# Cost of Plutocracy

The aim of this article is to analyze some of the biggest DAOs in the crypto space and see how many of them are influenced by token holders which own a disproportionate amount of tokens compared to most voters.

In [1]:
# sets up the pynb environment
import os
import sys

import pandas as pd
import numpy as np

from libs.data_processing.filters import get_quartile_by_vp


module_path = os.path.abspath(os.path.join("."))
if module_path not in sys.path:
    sys.path.append(module_path)

The data used for this article came from data sources that specialize in providing data on DAO governance, such as DeepDAO and Snapshot. Coingecko was also used to find market data for DAO native tokens.

Using this data, two spreadsheets were compiled which act as the local database for this analysis. Each spreadsheet contains each voter's choice and voting power for the last five proposals each DAO opened. **One spreadsheet filters out "whales"** which, in the context of this analysis, are voters of each proposal which have voting power **at or above the 95th percentile of voting power for that proposal**.

In [2]:
plutocracy_report_data = pd.read_excel(
    "./plutocracy_report.xls", sheet_name=None, engine="openpyxl"
)
plutocracy_report_data_filtered = pd.read_excel(
    "./plutocracy_report_filtered.xls", sheet_name=None, engine="openpyxl"
)

With this, we can begin asking ourselves some questions. Such as: **How many whales have voted in the last five proposals for each of the DAOs we analyzed?**

In [3]:
def get_number_of_whales_to_all_voters_ratio(
    dao_proposals: dict[str, pd.DataFrame],
    dao_proposals_filtered: dict[str, pd.DataFrame],
) -> list[dict[str, int]]:
    ratios = []
    for organization in dao_proposals.keys():
        all_proposals = dao_proposals[organization]
        try:
            all_proposals_filtered = dao_proposals_filtered[organization]
        except KeyError:
            continue
        top_shareholders_df = get_quartile_by_vp(all_proposals, 0.95)
        top_shareholders_addresses = top_shareholders_df["voter"]
        tally = {organization: [0, 0, 0, 0, 0]}

        tally[organization][0] = all_proposals.shape[0] - all_proposals_filtered.shape[0]
        tally[organization][1] = all_proposals.shape[0]
        tally[organization][2] = all_proposals_filtered["vp"].mean()
        tally[organization][3] = all_proposals.loc[
            lambda df: [voter in top_shareholders_addresses.values for voter in df["voter"]]
        ]["vp"].mean()
        tally[organization][4] = all_proposals["cost_per_vote"].mean()
        ratios.append(tally)

    return ratios


voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [4]:
pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    index=[list(result.items())[0][0] for result in voting_ratios],
    columns=["# of whales", "all voters", "avg vp for non-whales", "avg vp for whales", "avg cost of vote"],
)


,# of whales,all voters,avg vp for non-whales,avg vp for whales,avg cost of vote
ENS,5,100,5703.764495,1.918741e+05,14.254275
Lido,5,100,21512.421100,1.176723e+06,1.366854
Wonderland,5,100,1359.500484,4.062221e+05,28028.436822
Frax,5,89,157132.307947,4.778937e+06,10.086675
OlympusDAO,5,100,225.137797,1.111827e+04,55.119179
Decentraland,5,100,2330.269381,2.243233e+05,0.662245
Curve Finance,4,73,3860.980650,6.548465e+06,2.600384
Radicle,5,64,213089.903084,2.830379e+06,4.079941
Fei,5,100,535.901800,2.331334e+07,0.224155


Here we can clearly see that for the DAOs which we choose to analyze the top 5% of voters have a clear, disproportionate amount of voting power comparared to the average voter.

We can also gain some insight at the economic might that these whales hold for each DAO at the time of voting by taking a look at the average cost of each vote for each DAO's native token at the time these proposals were active.

In [8]:
def get_score_differences(
    dao_proposals: dict[str, pd.DataFrame],
    dao_proposals_filtered: dict[str, pd.DataFrame],
) -> list[dict[str, dict]]:
    diffrences: list[dict[str, dict]] = []
    for organization in dao_proposals.keys():
        diffrences.append({organization: dict()})
        organization_proposals = diffrences[-1][organization]
        all_proposals = [
            proposal
            for _, proposal in dao_proposals[organization].groupby(
                "proposal_id", sort=False
            )
        ]
        try:
            all_proposals_filtered = [
                proposal
                for _, proposal in dao_proposals_filtered[organization].groupby(
                    "proposal_id", sort=False
                )
            ]
        except KeyError:
            del diffrences[-1]
            continue

        for proposal, proposal_filtered in zip(all_proposals, all_proposals_filtered):
            proposal_id: str = proposal.iloc[0]["proposal_id"]
            organization_proposals[proposal_id] = [
                score - score_filtered
                for score, score_filtered in zip(
                    eval(proposal.iloc[0]["proposal_scores"]),
                    eval(proposal_filtered.iloc[0]["proposal_scores"]),
                )
            ]
    return diffrences

score_diffrences = get_score_differences(plutocracy_report_data, plutocracy_report_data_filtered)

In [10]:
score_diffrences_dfs = dict()

for score_diffrence in score_diffrences:
    for organization, data in score_diffrence.items():
        data: dict[str, list] = data
        items = data.items()
        score_diffrences_dfs[organization] = pd.DataFrame(
            [[score] for _, score in items],
            index=pd.Index(([proposal_id for proposal_id, _ in items]), name="Proposal ID"),
            columns=["score differences"],
        )
score_diffrences_dfs["ENS"]

,score differences
Proposal ID,
0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7,"[205394.8058600719, 0.0, 0.0]"
0xd7eff781be059513b5cd64d79e709abbbc653944c9a8c621dc051e7b42a405cb,"[205394.8058600719, 0.0, 0.0]"
0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b,"[0.0, 205394.80586007194, 0.0]"
0xdaff050d30c77fe469da7eaa5b5bae0cb892f91a89c119367e47faf370667240,"[119.4051523450762, 119.4051523450762, 119.405..."
0xa245dc7264ae072620434996e1c66d82c64cefc5a2edde5114d691a8559d0b5f,"[119.40515234495979, 119.4051523450762, 119.40..."
